In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [3]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
train_df.text.isnull().sum()

0

In [7]:
train_df.sample(10)

,id,keyword,location,text,target
3662,5212,fatality,playing soccer & eating pizza,@Jake_ADavis @FaTality_US we are cuddling righ...,0
1429,2061,casualty,"Rochelle, GA",Being able to stay out of work this week to ta...,0
7413,10606,wounded,USA,One man fatally shot another wounded on Vermon...,1
6023,8608,seismic,Somalia,Exploration takes seismic shift in Gabon to So...,0
5015,7154,mudslide,Wales,Hope Dorett's 'mudslide' cake wins?? #GBBO,0
5371,7663,panic,NaN,we was all just laughing and talking junk now ...,0
3407,4877,explode,Australia,It's cold and my head wants to explode.. The j...,0
3681,5236,fatality,New York,@Babybackreeve FATALITY!!!!!!!!!!,0
3377,4835,evacuation,EIU Chucktown/LaSalle IL,@Eric_Bulak @jaclynsonne @_OliviaAnn_ I was lo...,0
6338,9060,structural%20failure,San Francisco,[CLIP] Top-down coercion - The structural weak...,0


In [8]:
import string
from nltk.corpus import stopwords
def clean_text(text):
    STOPWRDS = stopwords.words('english')
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWRDS])

In [9]:
train_df.text.head()

0    Our Deeds are the Reason of this #earthquake M...
1               Forest fire near La Ronge Sask. Canada
2    All residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object

In [10]:
train_df['text'] = train_df.text.apply(clean_text)
train_df.text.head()

0         Deeds Reason earthquake May ALLAH Forgive us
1                Forest fire near La Ronge Sask Canada
2    residents asked shelter place notified officer...
3    13000 people receive wildfires evacuation orde...
4    got sent photo Ruby Alaska smoke wildfires pou...
Name: text, dtype: object

In [11]:
test_df['text'] = test_df.text.apply(clean_text)
test_df.text.head()

0                          happened terrible car crash
1    Heard earthquake different cities stay safe ev...
2    forest fire spot pond geese fleeing across str...
3                Apocalypse lighting Spokane wildfires
4               Typhoon Soudelor kills 28 China Taiwan
Name: text, dtype: object

In [12]:
count_vectorizer = feature_extraction.text.CountVectorizer()
example_train_vectors = count_vectorizer.fit_transform(train_df['text'][0:5])
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 36)
[[0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0]]


In [13]:
train_vectors = count_vectorizer.fit_transform(train_df['text'])
test_vectors = count_vectorizer.transform(test_df['text'])

In [14]:
train_vectors.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
clf = linear_model.RidgeClassifier()

In [16]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df['target'], cv=3, scoring = 'f1')
scores

array([0.59462425, 0.52008239, 0.58925831])

In [17]:
clf.fit(train_vectors, train_df['target'])

RidgeClassifier()

In [18]:
preds = clf.predict(test_vectors)
submission = pd.DataFrame(preds,columns = ['Target']).join(test_df['id'])
submission.to_csv('submission.csv', index = False)

In [19]:
submission

,Target,id
0,0,0
1,1,2
2,1,3
3,1,9
4,1,11
...,...,...
3258,1,10861
3259,1,10865
3260,1,10868
3261,1,10874
